# Example usage notebook of the DD-GAN repo

In [1]:
import time
import numpy as np
import sklearn.preprocessing
import tensorflow as tf

After installation of the ddgan package run the following line to import it

In [2]:
from ddgan import GAN
from ddgan import *

In [3]:
kwargs = {
    "nsteps" : 5,
    "ndims" : 5,
    "lmbda" : 10,
    "n_critic" : 5,
    "batches" : 10,
    "batch_size" : 20,
    "seed" : 143
}

gan = GAN(**kwargs)
gan.setup()
set_seed(gan.seed)

ndims_latent_input = 5

#csv_data = np.load('../data/processed/pod_coeffs_field_Velocity_onegrid.npy', )
csv_data = np.loadtxt('../data/processed/POD_coeffs_1_204_orig.csv', delimiter=',')
csv_data = np.float32(csv_data)
#csv_data = csv_data[0,:,:]ææææææææææáððððáþÞÆ
                    

print('type and shape (nPOD by nTrain) of POD coeffs from csv file', type(csv_data), csv_data.shape, csv_data.dtype)


looking for previous saved models
making new generator and critic
type and shape (nPOD by nTrain) of POD coeffs from csv file <class 'numpy.ndarray'> (5, 204) float32


In [4]:
nTrain = csv_data.shape[1]
nPOD = csv_data.shape[0]

csv_data = csv_data.T # nTrain by nPOD


In [5]:
scaling = sklearn.preprocessing.MinMaxScaler(feature_range=[-1,1])
csv_data = scaling.fit_transform(csv_data)

for icol in range(csv_data.shape[1]):
    print('min and max of col, ', icol ,' of csv_data:', np.min(csv_data[:,icol]), np.max(csv_data[:,icol]) )

min and max of col,  0  of csv_data: -1.0 1.0
min and max of col,  1  of csv_data: -1.0 1.0000001
min and max of col,  2  of csv_data: -1.0 1.0
min and max of col,  3  of csv_data: -1.0 1.0
min and max of col,  4  of csv_data: -1.0 1.0


In [6]:
nsteps = 5
t_begin = 0
t_end = nTrain - nsteps + 1
training_data = np.zeros((t_end, nPOD * nsteps), dtype=np.float32) # nTrain by nsteps*nPOD # 'float32' or np.float32

input_to_GAN = tf.random.normal([training_data.shape[0], ndims_latent_input])
input_to_GAN = input_to_GAN.numpy()

for step in range(nsteps):
    training_data[:,step*nPOD:(step+1)*nPOD] = csv_data[t_begin+step : t_end+step,:]
print('Shape of training data for the GAN', training_data.shape, training_data.dtype)

Shape of training data for the GAN (200, 25) float32


In [ ]:
t0 = time.time()
gan.learn_hypersurface_from_POD_coeffs(nPOD, input_to_GAN, training_data, ndims_latent_input)
t_train = time.time() - t0

beginning training
epoch: 0


In [ ]:
kwargs_opt = {
    "initial" : 5,
    "inn" : 10,
    "iterations" : 20,
    "optimizer_epochs" : 5000,
    "GAN" : gan
}


optimizer = Optimize(**kwargs_opt)

In [ ]:
Optimize.timesteps()